In [3]:
import pandas as pd
from datasets import load_dataset, Dataset
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, TextDataset
from sklearn.model_selection import train_test_split
import torch

Это датасет, содержащий вопросы и ответы касательно ментального здоровья. В итоге получится некий ChatBot Therapist🤗



In [4]:
dataset = load_dataset("Amod/mental_health_counseling_conversations")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
data = pd.DataFrame(dataset['train'])

In [6]:
train_data, eval_data = train_test_split(data, test_size=0.2)

In [10]:
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
tokenizer.pad_token = tokenizer.eos_token

tokenized_train_data = tokenizer(
    train_data['Context'].tolist(),
    text_pair=train_data['Response'].tolist(),
    return_tensors='pt',
    padding=True,
    truncation=True,
    max_length=512
)

tokenized_eval_data = tokenizer(
    eval_data['Context'].tolist(),
    text_pair=eval_data['Response'].tolist(),
    return_tensors='pt',
    padding=True,
    truncation=True,
    max_length=512
)

tokenized_train_data['labels'] = tokenized_train_data['input_ids'].clone()
tokenized_eval_data['labels'] = tokenized_eval_data['input_ids'].clone()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [8]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


train_dataset = CustomDataset(tokenized_train_data)
eval_dataset = CustomDataset(tokenized_eval_data)

In [15]:
training_args = TrainingArguments(
    output_dir="./dialoGPT_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)


In [16]:
trainer.train()

<ipython-input-8-d4f0af6a42e7>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,1.517000
1000,1.432500
1500,1.634700
2000,1.395800
2500,1.424700
3000,1.335700
3500,1.263300
4000,1.291400
4500,1.206900
5000,1.165100


<ipython-input-8-d4f0af6a42e7>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-d4f0af6a42e7>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-d4f0af6a42e7>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-d4f0af6a42e7>:9: UserWarning: To copy construct from a tens

TrainOutput(global_step=14050, training_loss=1.1182367005806377, metrics={'train_runtime': 5059.1828, 'train_samples_per_second': 5.552, 'train_steps_per_second': 2.777, 'total_flos': 7339693178880000.0, 'train_loss': 1.1182367005806377, 'epoch': 10.0})

In [17]:
trainer.save_model("./dioloGPT_mental_health_model")
tokenizer.save_pretrained("/dioloGPT_mental_health_tok")

('/dioloGPT_mental_health_tok/tokenizer_config.json',
 '/dioloGPT_mental_health_tok/special_tokens_map.json',
 '/dioloGPT_mental_health_tok/vocab.json',
 '/dioloGPT_mental_health_tok/merges.txt',
 '/dioloGPT_mental_health_tok/added_tokens.json',
 '/dioloGPT_mental_health_tok/tokenizer.json')

In [20]:
model_name = "./dioloGPT_mental_health_model"
tokenizer = AutoTokenizer.from_pretrained('/dioloGPT_mental_health_tok')
tokenizer.pad_token_id = tokenizer.eos_token_id
model_config = model.config
model_config.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(model_name, config=model_config)

In [26]:
while True:
    user_input = input("You say: ")
    if user_input.lower() == 'exit':
        print ('Exiting the bot')
        break
    else:

        tokenized_input = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True)
        input_ids = tokenized_input["input_ids"]
        attention_mask = tokenized_input["attention_mask"]


        output = model.generate(input_ids, attention_mask=attention_mask, max_length=1000, num_beams=5, no_repeat_ngram_size=2, do_sample=True, temperature=0.2)
        generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_response = generated_response.replace(user_input, '').strip()

    print("ChatBot:", generated_response)

You say: Hi! i had a big argument with my friend and now i feel sad and anxious. what should i do?
ChatBot: Hi there.  I'm glad you reached out. I think it's important to take a look inside and see what's going on. It sounds like you have a lot on your plates and want to express it in a way that feels right to you. This is a great first step, and I hope you are able to find a therapist who can help you sort through this. Good luck!
You say: And how do i deal with the anxiety? What can i do to make myself feel better?
ChatBot: I'm sorry you are experiencing anxiety. Anxiety is a very natural part of life, and it is something that happens in a few different ways throughout the day, so I would encourage you to seek out a therapist who specializes in anxiety disorders.
You say: I also feel a bit lonely after the argument. What can i do?
ChatBot: Loneliness is not something that we are used to feeling. We are human and often times feel lonely because of it.  Feeling alone can be a sign of d

Чатбот в целом неплохо отвечает на вопросы, но чаще всего разговор заканчиваетя тем, что он направляет человек к психологу (другие советы выдает редко, что связано с содержанием датасета). Учитыва этические вопросы, когда дело касается чатботов, дающих советы по метальному здоровью, я считаю что это даже хорошо, что он направляет всех к специалисту))
